In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from google.colab import drive
import nltk
nltk.download('stopwords')
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [2]:
#tweets to text
df = pd.read_csv('/content/drive/MyDrive/cuatrimestres/Noveno cuatri/Natural Language Preprocessing/Project/data.csv', encoding = "ISO-8859-1")


from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re

def scrub_words(text):
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
  
    #remove whitespace
    text=text.strip()
    return text
  
#Noise removal, stop word removal, normalizing?
def cleanString(s, special_chars = "\":,.@|ðÿœžðÿâœœïÿœžÿºÿÿœžÿµ"):
    for char in special_chars:
        s = s.replace(char, "")
    s = s.replace("\n", "")
    s = scrub_words(s)
    s = s.lower()
    tokenizer = TweetTokenizer()
    stop_words = set(stopwords.words('english'))
    cleaned_words = [w for w in tokenizer.tokenize(s) if w not in stop_words]
    return " ".join(cleaned_words)

def stemWords(sentence):
    stemmer, tokenizer = PorterStemmer(), TweetTokenizer()
    stemmed_words = [stemmer.stem(w) for w in tokenizer.tokenize(sentence)]
    return " ".join(stemmed_words)
    
def cleanFrame(frame):
    frame['clean_tweet'] = frame.Tweet.apply(cleanString)

cleanFrame(df)
df = df['clean_tweet'].iloc[0:10000]
df.to_csv(r'text.txt', header=None, index=None, sep=' ', mode='a')
df.head()

0            need paradigm apple music itunes link let
1    hbo go month solo month shared months shared h...
2    still use apple music listen embarrassing musi...
3    officially switched apple music spotify idek feel
4              apple music damn irritating bitch music
Name: clean_tweet, dtype: object

In [3]:
# Read, then decode for py2 compat.
text = open('text.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 719542 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

"need paradigm apple music itunes link let"
"hbo go month solo month shared months shared hbo max also available wts wtb lfs lfb lf looking netflix viki apple music wattpad premium hbo viu amazon viki grammarly tidal quillbot quizlet chegg coursehero


In [5]:
#lista = text.replace('"','').split("\n")

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

51 unique characters


In [7]:
lista = text.replace('"','').split("\n")
chars = tf.strings.unicode_split(lista, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'n', b'e', b'e', b'd', b' ', b'p', b'a', b'r', b'a', b'd', b'i', b'g',
  b'm', b' ', b'a', b'p', b'p', b'l', b'e', b' ', b'm', b'u', b's', b'i',
  b'c', b' ', b'i', b't', b'u', b'n', b'e', b's', b' ', b'l', b'i', b'n',
  b'k', b' ', b'l', b'e', b't']                                          ,
 [b'h', b'b', b'o', b' ', b'g', b'o', b' ', b'm', b'o', b'n', b't', b'h',
  b' ', b's', b'o', b'l', b'o', b' ', b'm', b'o', b'n', b't', b'h', b' ',
  b's', b'h', b'a', b'r', b'e', b'd', b' ', b'm', b'o', b'n', b't', b'h',
  b's', b' ', b's', b'h', b'a', b'r', b'e', b'd', b' ', b'h', b'b', b'o',
  b' ', b'm', b'a', b'x', b' ', b'a', b'l', b's', b'o', b' ', b'a', b'v',
  b'a', b'i', b'l', b'a', b'b', b'l', b'e', b' ', b'w', b't', b's', b' ',
  b'w', b't', b'b', b' ', b'l', b'f', b's', b' ', b'l', b'f', b'b', b' ',
  b'l', b'f', b' ', b'l', b'o', b'o', b'k', b'i', b'n', b'g', b' ', b'n',
  b'e', b't', b'f', b'l', b'i', b'x', b' ', b'v', b'i', b'k', b'i', b' ',
  b'a', b'p', b'p', 

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[18, 9, 9, 8, 2, 20, 5, 22, 5, 8, 13, 11, 17, 2, 5, 20, 20, 16, 9, 2, 17,
  25, 23, 13, 7, 2, 13, 24, 25, 18, 9, 23, 2, 16, 13, 18, 15, 2, 16, 9, 24],
 [12, 6, 19, 2, 11, 19, 2, 17, 19, 18, 24, 12, 2, 23, 19, 16, 19, 2, 17, 19,
  18, 24, 12, 2, 23, 12, 5, 22, 9, 8, 2, 17, 19, 18, 24, 12, 23, 2, 23, 12,
  5, 22, 9, 8, 2, 12, 6, 19, 2, 17, 5, 28, 2, 5, 16, 23, 19, 2, 5, 26, 5,
  13, 16, 5, 6, 16, 9, 2, 27, 24, 23, 2, 27, 24, 6, 2, 16, 10, 23, 2, 16,
  10, 6, 2, 16, 10, 2, 16, 19, 19, 15, 13, 18, 11, 2, 18, 9, 24, 10, 16, 13,
  28, 2, 26, 13, 15, 13, 2, 5, 20, 20, 16, 9, 2, 17, 25, 23, 13, 7, 2, 27,
  5, 24, 24, 20, 5, 8, 2, 20, 22, 9, 17, 13, 25, 17, 2, 12, 6, 19, 2, 26,
  13, 25, 2, 5, 17, 5, 30, 19, 18, 2, 26, 13, 15, 13, 2, 11, 22, 5, 17, 17,
  5, 22, 16, 29, 2, 24, 13, 8, 5, 16, 2, 21, 25, 13, 16, 16, 6, 19, 24, 2,
  21, 25, 13, 30, 16, 9, 24, 2, 7, 12, 9, 11, 11, 2, 7, 19, 25, 22, 23, 9,
  12, 9, 22, 19, 2, 6, 5, 22, 24, 16, 9, 6, 29, 2, 17, 23, 2, 17, 23, 2, 23,
 

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'n', b'e', b'e', b'd', b' ', b'p', b'a', b'r', b'a', b'd', b'i', b'g',
  b'm', b' ', b'a', b'p', b'p', b'l', b'e', b' ', b'm', b'u', b's', b'i',
  b'c', b' ', b'i', b't', b'u', b'n', b'e', b's', b' ', b'l', b'i', b'n',
  b'k', b' ', b'l', b'e', b't']                                          ,
 [b'h', b'b', b'o', b' ', b'g', b'o', b' ', b'm', b'o', b'n', b't', b'h',
  b' ', b's', b'o', b'l', b'o', b' ', b'm', b'o', b'n', b't', b'h', b' ',
  b's', b'h', b'a', b'r', b'e', b'd', b' ', b'm', b'o', b'n', b't', b'h',
  b's', b' ', b's', b'h', b'a', b'r', b'e', b'd', b' ', b'h', b'b', b'o',
  b' ', b'm', b'a', b'x', b' ', b'a', b'l', b's', b'o', b' ', b'a', b'v',
  b'a', b'i', b'l', b'a', b'b', b'l', b'e', b' ', b'w', b't', b's', b' ',
  b'w', b't', b'b', b' ', b'l', b'f', b's', b' ', b'l', b'f', b'b', b' ',
  b'l', b'f', b' ', b'l', b'o', b'o', b'k', b'i', b'n', b'g', b' ', b'n',
  b'e', b't', b'f', b'l', b'i', b'x', b' ', b'v', b'i', b'k', b'i', b' ',
  b'a', b'p', b'p', 

In [12]:
#de id a texto
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

text_from_ids(ids)

<tf.Tensor: shape=(10001,), dtype=string, numpy=
array([b'need paradigm apple music itunes link let',
       b'hbo go month solo month shared months shared hbo max also available wts wtb lfs lfb lf looking netflix viki apple music wattpad premium hbo viu amazon viki grammarly tidal quillbot quizlet chegg coursehero bartleby ms ms spotify solo',
       b'still use apple music listen embarrassing music without showing spotify wrapped',
       ...,
       b'wrong applemusic include dangerous one michaeljackson essential albums always bias toward early releases lists dangerous mid career masterpiece',
       b'fuck apple music working', b''], dtype=object)>

TRAINNING EXAMPLE

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(719542,), dtype=int64, numpy=array([ 3, 18,  9, ..., 11,  3,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

"
n
e
e
d
 
p
a
r
a


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'"' b'n' b'e' b'e' b'd' b' ' b'p' b'a' b'r' b'a' b'd' b'i' b'g' b'm'
 b' ' b'a' b'p' b'p' b'l' b'e' b' ' b'm' b'u' b's' b'i' b'c' b' ' b'i'
 b't' b'u' b'n' b'e' b's' b' ' b'l' b'i' b'n' b'k' b' ' b'l' b'e' b't'
 b'"' b'\n' b'"' b'h' b'b' b'o' b' ' b'g' b'o' b' ' b'm' b'o' b'n' b't'
 b'h' b' ' b's' b'o' b'l' b'o' b' ' b'm' b'o' b'n' b't' b'h' b' ' b's'
 b'h' b'a' b'r' b'e' b'd' b' ' b'm' b'o' b'n' b't' b'h' b's' b' ' b's'
 b'h' b'a' b'r' b'e' b'd' b' ' b'h' b'b' b'o' b' ' b'm' b'a' b'x' b' '
 b'a' b'l' b's'], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'"need paradigm apple music itunes link let"\n"hbo go month solo month shared months shared hbo max als'
b'o available wts wtb lfs lfb lf looking netflix viki apple music wattpad premium hbo viu amazon viki g'
b'rammarly tidal quillbot quizlet chegg coursehero bartleby ms ms spotify solo"\n"still use apple music '
b'listen embarrassing music without showing spotify wrapped"\n"officially switched apple music spotify i'
b'dek feel"\n"apple music damn irritating bitch music"\n"spotify gt apple music"\n"bite every apple music '


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(lista)

(['need paradigm apple music itunes link let',
  'hbo go month solo month shared months shared hbo max also available wts wtb lfs lfb lf looking netflix viki apple music wattpad premium hbo viu amazon viki grammarly tidal quillbot quizlet chegg coursehero bartleby ms ms spotify solo',
  'still use apple music listen embarrassing music without showing spotify wrapped',
  'officially switched apple music spotify idek feel',
  'apple music damn irritating bitch music',
  'spotify gt apple music',
  'bite every apple music user',
  'picsart gold month months ph wts wtb lfs lfb lf looking netflix shared viki standard plus apple music wattpad premium hbo viu amazon viki grammarly tidal duo go plus quillbot quizlet chegg coursehero bartleby microsoft ms spotify canva pro',
  'spotify users desperate claim superiority apple music enjoyers forget androids',
  'apple music replay',
  'apple music actually better music quality spotify',
  'hahaha yall apple music replay actually accurate',
  'min

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'"need paradigm apple music itunes link let"\n"hbo go month solo month shared months shared hbo max al'
Target: b'need paradigm apple music itunes link let"\n"hbo go month solo month shared months shared hbo max als'


#Create training batches

In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [26]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [27]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 52) # (batch_size, sequence_length, vocab_size)


In [28]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  13312     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  53300     
                                                                 
Total params: 4,004,916
Trainable params: 4,004,916
Non-trainable params: 0
_________________________________________________________________


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [30]:
sampled_indices

array([21, 25, 14, 50, 13, 20,  0, 12, 16, 48, 44, 40, 45, 35, 51, 43, 10,
       31, 18, 38,  5,  0, 45,  1, 20,  4, 28, 12, 11,  6, 18,  1, 21, 28,
        5, 33,  5, 17,  6,  9, 47, 46, 47,  9, 21, 43, 10, 33, 40, 38, 23,
        6, 10, 12, 31, 25, 11, 35, 45,  2, 41,  9,  0, 15, 36, 34, 51, 13,
       50, 26,  2,  3, 14, 10, 43, 37, 27, 40, 38, 36,  4, 36,  1, 44, 17,
        6, 32, 40,  4, 13,  8, 16,  4, 27, 29, 42,  3, 17, 14, 50])

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'evices"\n"ambient sound station apple music rn"\n"uses apple music"\n"dream smp list categories spotify'

Next Char Predictions:
 b'quj\xc3\xadip[UNK]hl\xc3\xab\xc3\xa6\xc3\xa2\xc3\xa7\xc2\xbc\xc3\xaf\xc3\xa5f\xc2\xaan\xc3\xa0a[UNK]\xc3\xa7\np_xhgbn\nqxa\xc2\xb3ambe\xc3\xaa\xc3\xa9\xc3\xaaeq\xc3\xa5f\xc2\xb3\xc3\xa2\xc3\xa0sbfh\xc2\xaaug\xc2\xbc\xc3\xa7 \xc3\xa3e[UNK]k\xc2\xbd\xc2\xb9\xc3\xafi\xc3\xadv "jf\xc3\xa5\xc2\xbew\xc3\xa2\xc3\xa0\xc2\xbd_\xc2\xbd\n\xc3\xa6mb\xc2\xb2\xc3\xa2_idl_wy\xc3\xa4"mj\xc3\xad'


# training model

In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 52)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(3.949756, shape=(), dtype=float32)


In [34]:
tf.exp(example_batch_mean_loss).numpy()

51.92269

In [52]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [53]:
model.compile(optimizer='adam', loss=loss,metrics=["accuracy"])

In [54]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [55]:
EPOCHS = 25

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
111/111 [==============================] - 708s 6s/step - loss: 1.9189 - accuracy: 0.4506
Epoch 2/25
111/111 [==============================] - 695s 6s/step - loss: 1.5544 - accuracy: 0.5539
Epoch 3/25
111/111 [==============================] - 704s 6s/step - loss: 1.3464 - accuracy: 0.6115
Epoch 4/25
111/111 [==============================] - 703s 6s/step - loss: 1.2213 - accuracy: 0.6437
Epoch 5/25
111/111 [==============================] - 717s 6s/step - loss: 1.1335 - accuracy: 0.6664
Epoch 6/25
111/111 [==============================] - 713s 6s/step - loss: 1.0631 - accuracy: 0.6841
Epoch 7/25
111/111 [==============================] - 694s 6s/step - loss: 1.0007 - accuracy: 0.7008
Epoch 8/25
111/111 [==============================] - 697s 6s/step - loss: 0.9404 - accuracy: 0.7174
Epoch 9/25
111/111 [==============================] - 694s 6s/step - loss: 0.8810 - accuracy: 0.7334
Epoch 10/25
111/111 [==============================] - 708s 6s/step - loss: 0.8210 - accura

# Make predictions


In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['I would prefer to'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

I would prefer today gonna add lsspob apple music apple music ½"
"lowkey genuinely scared see bujuers remix apple music ¹"
"looked apple music replay gonfa creat seemed songs number one album lol"
"apple music wrapped time um number roast time play music"
"apple replay groups fucked asium top"
"apple music catch years facebook industry bad"
"pay apple music wrapped songs dey bore get"
"bicycle spotify apple music apple music"
"bicycle apple music namjoonie"
"wizkid apple music dropped bc listen music dicks ¹"
"artists say album apple music wrapped spotify users get festival use fillers"
"played song apple music experienced phone play blatter"
"faturf spotify wrapped using apple music browser checked nigerian asake number song apple music replay"
"apple music replay look consistent yeat top global dreamers bts_twt dreamersbyjungkook"
"ok applemusic youtubemusic added world good breath today"
"post apple music rewind fun"
"anyone music recommended worst theg bc stone marvin room remind u

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Shopify looks like ', 'Apple music is ', 'I think it is ', 'I would prefer '])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
for i in result:
    print(i)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(b'Shopify looks like followed"\n"anyone else apple music going \xc3\xa2 flaming ridit cuz call everyday every nan qodur sooooo \xc2\xbe"\n"finally put switch apple music fight back spotifycheam spotify replay"\n"apple music replay layout year randomly dutter"\n"way apple music replays cute shit like spotify"\n"spotify solow came unreleased songs apple music dirtwdack una lerry google disco machine need amp jyn part sound tracks plus app idk work"\n"apple music users real side number one tate discover"\n"ah yes time year wild apple music love collection everywhere know canceling spotify users stay trappic"\n"spotify wrapped apple music \xc2\xbd \xc2\xb2 \xc2\xb2"\n"listening get bulbie bit well make use spotify time discover songs apple music wattpad premium account hbo viu amazon viki commonti bew hay maces push bryathe put frads borda sa apple music pob tbh"\n"certain somebody spanis barbe apple music zonauang"\n"lmv top apple music copied spotify users feels songs pounty ja

In [ ]:
tf.saved_model.save(one_step_model, '/content/drive/MyDrive/cuatrimestres/Noveno cuatri/Natural Language Preprocessing/Project/one_step_2')


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from google.colab import drive
import nltk
nltk.download('stopwords')
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The actual model

In [ ]:
one_step_reloaded = tf.saved_model.load('/content/drive/MyDrive/cuatrimestres/Noveno cuatri/Natural Language Preprocessing/Project/one_step')

In [ ]:
states = None
next_char = tf.constant(['I really hate this '])
result = [next_char]

for n in range(1500):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

I really hate this seamers pob pc pob reppar apple music tonaget"
"finally put version contrach legal covers remembered used ditched apple music chart best metasi rusirious resouncedland resorrently bitch always website get apple tv workout clear"
"friend paradigm still available ups create face album na sa apple music wow"
"fat pls buy mon game soundtracks apple music rn"
"tiã sto business hiphop charts dance via applemusic"
"matter got phone less want year get jestival thing apple music new feature spotify playlist obts applemusic release early spotify"
"thoughts applemusic people send fun"
"music somehow amp curated playon ine sorry apple music sooo soon"
"spotify wrapped apple music elongunk pooo radio shit"
"spotify instafest ghettop ferrely lossess release get time stop couch moving apple music"
"really really enjoy find apple music create bicycle block audio amp make look like"
"apple music x bts know imp"
"audio quality never since spotify thoughts apple music dey stan guy abeg